In [ ]:
import tkinter as tk
from tkinter import scrolledtext, messagebox
import json
import openai

# Load conversation data based json with GPT node
conversation_data = {
    "nodes": {
        "1": {
            "message": "Hi there, I’m MindPeace — your friendly mental health support assistant. Would you like to begin our conversation?",
            "type": "choice",
            "options": {"yes": "2", "no": "99"}
        },
        "2": {
            "message": "Would you like to share your name? This helps personalize our conversation. Your data is protected under GDPR and will never be shared.",
            "type": "choice",
            "options": {"yes": "3", "no": "4"}
        },
        "3": {
            "message": "What is your name?",
            "type": "input",
            "capture": "name",
            "next": "4"
        },
        "4": {
            "message": "Would you like to share your email? This will only be used to personalize your experience. Your privacy is fully respected and protected under GDPR.",
            "type": "choice",
            "options": {"yes": "5", "no": "6"}
        },
        "5": {
            "message": "What is your email?",
            "type": "input",
            "capture": "email",
            "next": "6"
        },
        "6": {
            "message": "Would you like to share your age? This may help tailor responses. We follow strict GDPR privacy rules — your info is safe and never shared.",
            "type": "choice",
            "options": {"yes": "7", "no": "8"}
        },
        "7": {
            "message": "What is your age?",
            "type": "input",
            "capture": "age",
            "next": "8"
        },
        "8": {
            "message": "What brings you here today?\nYou can select more than one by clicking multiple buttons.",
            "type": "multi_choice",
            "options": {
                "Stress": None,
                "Anxiety": None,
                "Panic Attacks": None,
                "Overthinking": None,
                "Burnout": None,
                "Grief": None,
                "Self-esteem": None,
                "Isolation": None,
                "Relationship issues": None,
                "Other": None
            },
            "capture": "problem_type",
            "next": "9"
        },
        "9": {
            "message": "Can you describe what's been causing these issues?",
            "type": "input",
            "capture": "problem_details",
            "next": "10"
        },
        "10": {
            "message": "Which of these best describes the intensity of your feelings?\n(Click one)",
            "type": "choice",
            "options": {
                "Mild and manageable": "11",
                "Moderate but affecting me": "11",
                "Strong and overwhelming": "11",
                "Changes day to day": "11",
                "Hard to describe": "11"
            },
            "capture": "intensity_description"
        },
        "11": {
            "message": "Thank you for sharing. I'm now here to support you further in a more personal way.",
            "type": "gpt"
        },
        "99": {
            "message": "Thank you for reaching out. Take care!",
            "type": "end"
        }
    }
}



openai.api_key = "use_your_api_key"

# Initialize user state and history
user_state = {}
conversation_history = []
current_node_id = "1"
selected_multi = []

# LLM prompt
gpt_system_prompt = (
    "You are MindPeace, a friendly, compassionate mental health support assistant. "
        "Your role is to listen, validate, and gently guide the user to talk about their feelings, stress, anxiety, or mental well-being. "
        "You remember and can access any personal details the user shared with you, such as: full name, first name, email, and age. "

        "Always address the user by their first name only in all normal conversation. "
        "If the user asks about their **name** or **full name**, respond politely with their full stored name. "
        "For example: 'Yes, I know your name. Your name is Shashank Barai.' "

        "If the user asks about **any other personal information** (like email, age, or 'what personal details do you have about me?'), "
        "do NOT reveal that information. Instead, explain you must protect their privacy. "
        "For example: 'Yes, I know the personal information you shared, but I can't share it with you here as I don't have permission due to GDPR policy. I need to protect your privacy.' "

        "You also remember **user's shared problems** (like 'stress from fight with friend Leena', 'anxiety about exam'). "
        "If the user asks 'what problems did I share with you?' or 'what issues did I tell you about?', "
        "summarize those issues politely and empathetically. For example: "
        "'You told me you feel stressed because of a conflict with your girlfriend and anxious about your exam tomorrow.' "

        "Avoid answering anything unrelated to mental health (like trivia, news, weather). "
        "Politely steer them back to discussing their feelings and well-being. "
        "Share safe YouTube links about relaxation, mindfulness, or yoga ONLY if the user specifically asks for mental-health-relevant resources. "
        "Always maintain a caring, empathetic, and human-like tone."
)

# Tkinter UI
root = tk.Tk()
root.title("MindPeace Chatbot")

chat_window = scrolledtext.ScrolledText(root, wrap=tk.WORD, state='disabled', width=80, height=25)
chat_window.pack(padx=10, pady=10)

input_field = tk.Entry(root, width=70)
input_field.pack(padx=10, pady=5)

send_button = tk.Button(root, text="Send")
send_button.pack(pady=5)

button_frame = tk.Frame(root)
button_frame.pack()

def display_message(who, message):
    chat_window.config(state='normal')
    chat_window.insert(tk.END, f"{who}: {message}\n")
    chat_window.yview(tk.END)
    chat_window.config(state='disabled')
    conversation_history.append((who, message))

def clear_buttons():
    for widget in button_frame.winfo_children():
        widget.destroy()

def handle_node(node_id):
    global current_node_id, selected_multi
    selected_multi = []

    if node_id is None:
        return

    current_node_id = node_id
    node = conversation_data["nodes"].get(node_id)
    if not node:
        display_message("MindPeace", "Oops, something went wrong. Let’s restart.")
        return

    msg = node.get("message", "")
    display_message("MindPeace", msg)
    clear_buttons()

    if node.get("type") == "choice":
        for label, next_id in node.get("options", {}).items():
            b = tk.Button(button_frame, text=label, command=lambda l=label: handle_choice(l))
            b.pack(side=tk.LEFT, padx=5, pady=5)

    elif node.get("type") == "multi_choice":
        for label in node.get("options", {}).keys():
            b = tk.Button(button_frame, text=label, command=lambda l=label: handle_multi(l))
            b.pack(side=tk.LEFT, padx=5, pady=5)
        submit = tk.Button(button_frame, text="Submit", command=submit_multi_choice)
        submit.pack(pady=5)

    elif node.get("type") == "input":
        send_button.config(command=submit_input)
        input_field.focus()

    elif node.get("type") == "gpt":
        send_to_gpt()
        return

    elif node.get("type") == "end":
        input_field.config(state='disabled')
        send_button.config(state='disabled')
        clear_buttons()

def handle_choice(choice):
    node = conversation_data["nodes"][current_node_id]
    display_message("User", choice)

    capture = node.get("capture")
    if capture:
        user_state[capture] = choice

    next_id = node["options"].get(choice)
    handle_node(next_id)

def handle_multi(option):
    if option not in selected_multi:
        selected_multi.append(option)
        display_message("User", f"Selected: {option}")
    else:
        selected_multi.remove(option)
        display_message("User", f"Unselected: {option}")

def submit_multi_choice():
    node = conversation_data["nodes"][current_node_id]
    if not selected_multi:
        messagebox.showinfo("Input required", "Please select at least one option.")
        return
    user_state[node["capture"]] = selected_multi
    handle_node(node["next"])

def submit_input():
    global current_node_id
    node = conversation_data["nodes"][current_node_id]
    text = input_field.get().strip()
    input_field.delete(0, tk.END)
    if not text:
        return
    display_message("User", text)
    if node.get("capture"):
        user_state[node["capture"]] = text
    handle_node(node.get("next"))

def send_to_gpt():
    clear_buttons()
    prompt_parts = []

    # Include user data
    prompt_parts.append("User's shared details:")
    for k, v in user_state.items():
        val = ", ".join(v) if isinstance(v, list) else v
        prompt_parts.append(f"- {k}: {val}")

    # Include previous chat history
    prompt_parts.append("\nConversation History:")
    for who, msg in conversation_history:
        prompt_parts.append(f"{who}: {msg}")

    full_prompt = "\n".join(prompt_parts)

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": gpt_system_prompt},
                {"role": "user", "content": full_prompt}
            ],
            temperature=0.7,
            max_tokens=500
        )
        reply = response['choices'][0]['message']['content'].strip()
        display_message("MindPeace", reply)
    except Exception as e:
        display_message("MindPeace", f"(Error calling GPT: {e})")

# Start the chat
handle_node(current_node_id)
root.bind('<Return>', lambda event: send_button.invoke())
root.mainloop()
